In [1]:
import time, random, re, urllib.parse
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, StaleElementReferenceException
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from getpass import getpass
from selenium.webdriver.common.keys import Keys

/Users/zeyuhan/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


ModuleNotFoundError: No module named 'webdriver_manager'

In [2]:
import sys
import os

# 设置工作路径为当前notebook所在目录
notebook_dir = os.getcwd()
if notebook_dir not in sys.path:
    sys.path.append(notebook_dir)

In [32]:
from crawler import linkedin_common_crawler, login_linkedin
from url_generator import generate_urls
from cookies import save_cookies, load_cookies

In [4]:
urls = generate_urls()

Generating 270 combinations...
Successfully generated 270 URLs.


In [5]:
from concurrent.futures import ThreadPoolExecutor

max_workers = 10
executor = ThreadPoolExecutor(max_workers=max_workers)

print(f"Pool is created, max pool: {max_workers}")

线程池已创建，最大线程数: 10


In [6]:
from queue import Queue

# 创建一个队列并将urls加入队列
url_queue = Queue()
for url in urls:
    url_queue.put(url)

print(f"已将 {url_queue.qsize()} 个URL加入队列")

已将 270 个URL加入队列


In [28]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

In [ ]:

driver.get("https://www.linkedin.com/")
load_cookies(driver)

Cookies 已载入


In [ ]:
def crawl_url(url):
    
    try:
        result = linkedin_common_crawler(url)
        print(f"爬取成功: {url}")
        return result
    except Exception as e:
        print(f"爬取失败: {url}, 错误: {e}")
        return None

futures = []
while not url_queue.empty():
    url = url_queue.get()
    future = executor.submit(crawl_url, url)
    futures.append(future)

for future in futures:
    data = future.result()
    if data is not None:
        results.append(data)

print(f"共爬取成功 {len(results)} 个URL")

In [33]:
import threading
import queue
import time

# 待爬取 URL 队列
url_queue = queue.Queue()

# 全局结果列表（线程安全写入时最好加锁）
results = []
results_lock = threading.Lock()

def init_driver(cookies_file="cookies.pkl"):
    driver = webdriver.Chrome()
    driver.get("https://www.linkedin.com")  # 必须先打开域名才能加 cookie

    # 载入 cookie
    if load_cookies(driver, cookies_file):
        driver.refresh()
        time.sleep(3)  # 等待页面刷新完成
    else:
        print("没有 cookies，需要人工登录")
        # 如果第一次跑，需要人工登录并保存 cookie
        driver.get("https://www.linkedin.com/login")
        input("请在浏览器中登录后按回车继续...")
        save_cookies(driver, cookies_file)
        driver.refresh()
        time.sleep(3)

    return driver


def worker(worker_id, url_queue, cookies_file="cookies.pkl"):
    driver = init_driver(cookies_file)
    while True:
        try:
            url = url_queue.get(timeout=5)
        except queue.Empty:
            break

        try:
            data = linkedin_common_crawler(driver, url)
            if data:
                with results_lock:
                    results.append(data)
                print(f"[Worker {worker_id}] OK: {url}, got {len(data['jobs'])} jobs")
            else:
                print(f"[Worker {worker_id}] No data: {url}")
        except Exception as e:
            print(f"[Worker {worker_id}] Error on {url}: {e}")
        finally:
            url_queue.task_done()

    driver.quit()
    print(f"[Worker {worker_id}] finished")


def run_crawler(urls, num_workers=3):
    """运行爬虫调度"""
    for url in urls:
        url_queue.put(url)

    threads = []
    for i in range(num_workers):
        t = threading.Thread(target=worker, args=(i, url_queue))
        t.start()
        threads.append(t)

    # 等待所有任务完成
    url_queue.join()

    # 等待所有线程退出
    for t in threads:
        t.join()

    return results


In [35]:
run_crawler(urls[:20], num_workers=3)

Cookies 已载入
Cookies 已载入
Cookies 已载入
[Worker 2] OK: https://www.linkedin.com/jobs/search/?keywords=data+center&location=United+States&sortBy=R&f_E=1&f_WT=1&f_TPR=None&f_SB2=1, got 25 jobs
[Worker 1] OK: https://www.linkedin.com/jobs/search/?keywords=data+center&location=United+States&sortBy=R&f_E=1&f_WT=1&f_TPR=None&f_SB2=2, got 25 jobs
[Worker 2] OK: https://www.linkedin.com/jobs/search/?keywords=data+center&location=United+States&sortBy=R&f_E=1&f_WT=1&f_TPR=None&f_SB2=4, got 27 jobs
[Worker 1] OK: https://www.linkedin.com/jobs/search/?keywords=data+center&location=United+States&sortBy=R&f_E=1&f_WT=1&f_TPR=None&f_SB2=5, got 25 jobs
[Worker 0] OK: https://www.linkedin.com/jobs/search/?keywords=data+center&location=United+States&sortBy=R&f_E=1&f_WT=1&f_TPR=None&f_SB2=3, got 33 jobs
[Worker 2] OK: https://www.linkedin.com/jobs/search/?keywords=data+center&location=United+States&sortBy=R&f_E=1&f_WT=1&f_TPR=r604800&f_SB2=1, got 25 jobs
[Worker 1] OK: https://www.linkedin.com/jobs/search/?ke

[{'url': 'https://www.linkedin.com/jobs/search/?keywords=data+center&location=United+States&sortBy=R&f_E=1&f_WT=1&f_TPR=None&f_SB2=1',
  'jobs': [{'job_id': '4227919269',
    'job_name': 'Robot Optics Intern',
    'company_name': 'Neuralink',
    'job_location': 'Fremont, CA (On-site)',
    'job_metadata': '$35/hr · Vision, +1 benefit',
    'job_url': 'https://www.linkedin.com/jobs/view/4227919269/?eBP=CwEAAAGZfU4izJrwLOrLnNKB7euLkSTJ3gs8ANO7sak5LML_yP1kpWrwlWxiLKxrIZjnBShmKjdSGTxBn5PRFI2AJjSEYs7qYKWQG_mQc2mr2vMV-2NY5tDUWw8Wo_GdUWjZqAZO56TZPPlDyCXseeIfC6TA7Xg91GO2lHZtM3fQ1ZTAjs5rMuEBbYiy68F4g8CZ61chSJUgiO9cASmD7yynQd-taTDylySR4i2X8yvzCFdV_YvHVdEWrLk2FIrQG3q6NLIu7tptxtXk0shRFN8fB7wofzeFIFzGzQ6wpvOabrc_DZNi1tQoIFxESIftTwmy5-Z0UeeHLPrpXnJywxgxeN6wS2K9QguzInyftD4V5jLr25aK2sXA-Nza1xOiT6xYjwKh7R8_iFFHwiCXBPkaioj-I8jiU-oInISCnz62yy4vnpomWuQ_UtuLNWDgd71MXgklw_LtlI-pM8wDjcQV1vonzD3mKN--aON34w&refId=QLWtAzro1QP%2BdgKhFifOiw%3D%3D&trackingId=1dqfU5bMly9jHcWSynBBuw%3D%3D&trk=flagship3_search_srp_j

In [38]:
len(results[0])

2

In [45]:
l = []
for i in results:
    l.extend(i['jobs'])


In [46]:
len(l)

395